In [ ]:
!pip install transformers datasets --quiet

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [ ]:
def load_dataset(file_path, tokenizer, block_size=256):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

In [ ]:
train_dataset = load_dataset("output.txt", tokenizer)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=40,
    prediction_loss_only=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

In [ ]:
print(len(train_dataset))

15


In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
40,2.264800
80,1.259600
120,0.879600


TrainOutput(global_step=120, training_loss=1.4680244127909343, metrics={'train_runtime': 27.5096, 'train_samples_per_second': 8.179, 'train_steps_per_second': 4.362, 'total_flos': 29395353600000.0, 'train_loss': 1.4680244127909343, 'epoch': 15.0})

In [ ]:
trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")

('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [ ]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

In [ ]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [ ]:
import re
def postprocess_news_output(text):
    text = text.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")

    text = re.sub(r"</?ref>", "", text)
    text = re.sub(r"\*+", "", text)

    text = re.sub(r"retailers\'s", "retailers'", text)

    text = re.sub(r"\s+", " ", text).strip()

    sentences = re.split(r'(?<=[.?!])\s+', text)
    if not sentences[-1][-1] in [".", "!", "?"]:
        sentences = sentences[:-1]

    return " ".join(sentences)


In [ ]:
prompt="The man jumped on horizon...."

In [ ]:
raw_output = generator(
    prompt,
    max_new_tokens=180,
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
    )[0]['generated_text']
cleaned_output = postprocess_news_output(raw_output)
print(cleaned_output)

The man jumped on horizon.... "AHHHHH!!" shouted the pilot. The ship was engulfed in a massive fireball, and its crew were consumed with emotion as they watched helplessly from space...and then lost their minds! A powerful signal had been detected floating above Earth that hinted at an impending disaster upon completion! Captain Zarae Torres commanded her team to prepare for the event while ensuring it never occurred again!! As she prepared, Commander Riku Nakamura led them through various stages of preparations including deep-space reentry studies; advanced navigation systems enhancements designed to prevent catastrophic events like these--while maintaining stability throughout the journey.


In [ ]:
#output.txt contains:

In [ ]:
# #The Last Starship
# In the year 3050, humanity’s last starship, *Endeavour*, sailed through the dark void of space. The once bustling Earth was now a distant memory, consumed by environmental collapse. The crew, led by Captain Aria Nelson, was on a mission to find a new home. One day, as the ship scanned distant star systems, an ancient alien signal emerged from a forgotten corner of the galaxy. The signal, a cryptic message, hinted at a star system with planets capable of supporting life. Excitement filled the crew as they set course for the coordinates. Upon arrival, they discovered a solar system with three habitable planets. The crew began detailed scans and found an ancient archive buried beneath the surface of the third planet. The archive contained records of a civilization that had mastered space travel but chose to leave the system millennia ago. As the crew decoded the messages, they learned that the alien race had left behind advanced terraforming technology and detailed instructions for creating a new home. However, they also discovered a warning: a catastrophic event that had forced the aliens to abandon their home. Captain Nelson and her crew faced a choice. They could use the technology to settle on the planets, but they needed to prepare for the impending disaster described in the archives. The crew worked tirelessly to implement the alien technology, making the planets habitable and ensuring they were ready for the cataclysm. In the end, the *Endeavour* became a symbol of hope. Humanity had found a new home and was prepared to face the challenges ahead, thanks to the legacy of an ancient alien civilization.

# The Time Weaver
# Dr. Elara Voss, a brilliant physicist, had just completed her greatest invention: the Temporal Manipulator. This machine could alter time on a minuscule scale, affecting events within a small temporal window. Elara was excited to test her invention, eager to see its potential. During her first experiment, she made a minor adjustment to a single event. To her dismay, the results were far from what she expected. The minor change caused significant ripples, creating inconsistencies in the timeline. Historical records began to shift, and reality itself started to unravel. Elara realized that her experiment had inadvertently created a paradox. Desperate to fix the timeline, she sought the help of her assistant, Tom. Together, they analyzed the anomalies and discovered a more troubling issue: an alternate version of Elara from a different timeline was also trying to correct the paradox. Meeting her alternate self was both enlightening and disconcerting. The two Elaras had different approaches to resolving the timeline but shared a common goal. They combined their knowledge and skills, working together to stabilize the temporal distortions. After intense effort, they managed to restore the timeline to its original state. The Temporal Manipulator was shut down, and its technology was secured to prevent further misuse. Elara and her alternate self parted ways, their shared experience leaving them with a deeper understanding of time and the delicate balance required to maintain it.

# Echoes of Andromeda
# Astronomer Dr. Ravi Patel was thrilled when he detected a signal from the Andromeda galaxy. The signal, a complex pattern of electromagnetic waves, indicated the presence of an intelligent alien species. The news spread quickly, and a mission was organized to establish contact. Dr. Patel and his team traveled across the vast expanse of space in their starship, *Aurora*. When they arrived at the source of the signal, they discovered a planet with an advanced civilization. The aliens communicated through musical patterns, creating an intricate symphony that conveyed information. The team faced the challenge of understanding the alien language, which was entirely different from any form of communication they had encountered. With the help of advanced translation technology and a musical expert on board, they gradually deciphered the messages. The aliens revealed that they had been listening to Earth’s broadcasts for centuries, learning about human culture. They expressed a desire to establish a peaceful relationship and shared their knowledge of advanced technology. The exchange marked the beginning of a new era of intergalactic diplomacy, with both species learning from each other and working together to explore the cosmos.

# The Quantum Librarian
# In the year 2200, the Quantum Library was the pinnacle of human knowledge storage. Information was encoded in quantum states, accessible instantly across the galaxy. Mia Torres, a dedicated librarian, managed the library's vast archives. One day, while conducting routine maintenance, Mia discovered an unauthorized access point hidden deep within the library's system. Curiosity led her to a concealed section containing forbidden knowledge about ancient civilizations and lost technologies. As she delved deeper, Mia realized that this knowledge had been hidden for a reason. The revelations contained within the hidden archives had the potential to alter the course of history and challenge existing beliefs. Mia faced a dilemma: reveal the secrets and risk upheaval, or keep the knowledge concealed to preserve stability. She chose to share the information with a select group of scholars and leaders, ensuring that it was approached with caution and responsibility. The discovery led to new advancements and a re-evaluation of humanity's place in the galaxy, demonstrating the delicate balance between knowledge and wisdom.

# The Celestial Voyager
# Captain Marcus Steele commanded the *Celestial Voyager*, an experimental starship designed for deep-space exploration. On a routine mission to chart uncharted regions of the galaxy, the ship encountered a massive, ancient structure floating in space. The structure, resembling a colossal ring, was covered in intricate symbols and pulsating with a mysterious energy. As the crew investigated, they discovered that the ring was a gateway to a parallel dimension. The dimension was inhabited by a race of beings known as the Aetherians, who had mastered interdimensional travel. The Aetherians welcomed the crew and offered to share their knowledge and technology. However, the crew learned that the Aetherians were facing a dire threat from a rogue faction within their own dimension. The faction sought to harness the gateway's power for destructive purposes. Captain Steele and his crew allied with the Aetherians to prevent the faction's plans from succeeding. Their joint efforts led to a climactic battle to protect the gateway and ensure that the Aetherians could continue their peaceful existence. The *Celestial Voyager* returned to its dimension with new allies and a greater understanding of the multiverse.

# The Nebula Code
# In the year 2800, space travel had become routine, and humanity had established colonies across the galaxy. Dr. Zara Lee, an astrophysicist, was studying a massive nebula known as the Nebula Code

# The Forgotten Planet
# In the distant future, interstellar travel had become commonplace, but some regions of space remained unexplored. An expedition team led by Captain Lena Hart discovered a forgotten planet that had been lost to history. The planet, covered in lush forests and ancient ruins, showed signs of an advanced civilization that had mysteriously vanished. The team uncovered artifacts and writings that hinted at the planet's history and its inhabitants' advanced knowledge. As they explored, they realized that the civilization had developed a technology to transcend physical form and exist as energy beings. The team encountered remnants of these beings, who revealed their desire to share their knowledge with humanity. The beings offered insights into energy manipulation and sustainable living, enriching human understanding of the universe. The discovery of the forgotten planet became a milestone in human exploration, demonstrating the potential for ancient wisdom to shape the future.

# The Cosmic Architects
# A team of cosmic architects was tasked with designing a new galaxy to accommodate expanding civilizations. Using advanced technology, they created stars, planets, and ecosystems, carefully balancing the elements to ensure stability. As they worked on their grand project, they encountered unexpected challenges, including cosmic anomalies and competing interests from various civilizations. The architects had to collaborate and innovate to address these issues while maintaining the integrity of their designs. Their efforts culminated in the creation of a thriving galaxy, filled with diverse worlds and harmonious ecosystems. The project became a testament to the creativity and ingenuity of the cosmic architects, inspiring future generations to explore and appreciate the wonders of the universe.

# Celestial Architects
# The Celestials were an ancient alien race known for their artistry in designing and constructing planets. Each world they created was a masterpiece, reflecting their advanced technology and aesthetic vision. When Dr. Amelia Carter, an astrophysicist, discovered evidence of Celestial activity, she was amazed. The evidence revealed a hidden workshop where the Celestials crafted planets with intricate designs and ecosystems. Dr. Carter and her team were invited to explore the workshop and learn about the Celestials’ methods. They discovered that the Celestials had designed a new universe, with humanity as its central element. The architects had intended for this new universe to be a haven for sentient beings. As Dr. Carter and her team studied the designs, they realized that the Celestials had created a cosmic plan to bring harmony and balance to the universe. The discovery inspired humanity to continue exploring and appreciating the beauty of the cosmos. The Celestials’ legacy lived on through their creations, influencing future generations to seek beauty and harmony in their own endeavors.

# The Galactic Heist
# A notorious group of space thieves, known as the Nebula Shadows, planned their most daring heist yet: to steal a powerful artifact from the heart of a galactic fortress. The artifact, known as the Stellar Core, was said to possess immense power capable of reshaping entire star systems. The leader of the Nebula Shadows, Raxar, assembled a crew of skilled individuals, each with a unique talent. Their plan involved infiltrating the fortress, bypassing its advanced security systems, and extracting the Stellar Core without leaving a trace. As the heist unfolded, the crew faced unexpected obstacles, including rival factions and security enhancements. Their teamwork and ingenuity were tested as they navigated the fortress’s labyrinthine corridors and avoided detection. In the end, they succeeded in their mission, but the theft of the Stellar Core had far-reaching consequences. The artifact’s disappearance triggered a galactic search, leading to a new era of vigilance and intrigue in the galaxy. The Nebula Shadows' heist became legendary, and their story was told across the stars as a testament to their daring and skill.

# The Quantum Librarian
# In the year 2200, the Quantum Library was the pinnacle of human knowledge storage. Information was encoded in quantum states, accessible instantly across the galaxy. Mia Torres, a dedicated librarian, managed the library's vast archives. One day, while conducting routine maintenance, Mia discovered an unauthorized access point hidden deep within the library's system. Curiosity led her to a concealed section containing forbidden knowledge about ancient civilizations and lost technologies. As she delved deeper, Mia realized that this knowledge had been hidden for a reason. The revelations contained within the hidden archives had the potential to alter the course of history and challenge existing beliefs. Mia faced a dilemma: reveal the secrets and risk upheaval, or keep the knowledge concealed to preserve stability. She chose to share the information with a select group of scholars and leaders, ensuring that it was approached with caution and responsibility. The discovery led to new advancements and a re-evaluation of humanity's place in the galaxy, demonstrating the delicate balance between knowledge and wisdom.

# The Void Travelers
# In the far future, humanity had mastered the art of interdimensional travel. A team of scientists, led by Dr. Elena Wright, embarked on a mission to explore the multiverse. They used their advanced technology to navigate through various dimensions, each with its own unique properties. During their travels, they encountered parallel versions of Earth, some thriving and others in chaos. The team studied the different realities, learning valuable lessons about the nature of existence and the impact of choices. However, their journey was interrupted when they stumbled upon a dimension where an existential threat loomed. A powerful entity known as the Void was consuming entire dimensions, threatening the stability of the multiverse. Dr. Wright and her team had to find a way to stop the Void and save the dimensions from destruction. Their mission became a race against time, requiring them to use their knowledge and technology to combat the threat. In the end, they succeeded in halting the Void's advance, but their experiences left them with a deeper understanding of the multiverse's complexity and the interconnectedness of all realities.

# The Stellar Gardeners
# In a distant future, humanity had spread across the galaxy and established colonies on various planets. The Stellar Gardeners were a specialized group of scientists and engineers responsible for terraforming and cultivating new worlds. Their mission was to create habitable environments and ensure the growth of diverse ecosystems. One day, they were assigned a challenging task: to terraform a barren planet with extreme conditions. The team faced numerous obstacles, including hostile weather patterns and toxic atmospheres. Using their advanced technology and innovative techniques, they managed to transform the planet into a thriving paradise. However, their success attracted the attention of a rival organization that sought to exploit the planet's resources. The Stellar Gardeners had to defend their creation and ensure that the planet remained a sanctuary for life. Their efforts resulted in a peaceful resolution, and the planet became a symbol of hope and collaboration. The Stellar Gardeners' work demonstrated the potential for humanity to create and protect new worlds.

# The Galactic Mapmakers
# The Galactic Mapmakers were a group of explorers and cartographers responsible for charting the uncharted regions of the galaxy. Their work was crucial for navigation and the expansion of human civilization. One day, they discovered a mysterious anomaly in the fabric of space-time. The anomaly appeared to be a rift leading to an unknown region of the galaxy. The team decided to investigate and found a hidden star system with advanced civilizations and ancient technology. As they explored the system, they encountered beings who had been isolated for eons, unaware of the galaxy's progress. The Galactic Mapmakers established contact and learned about the beings' history and their advanced knowledge. They helped the beings integrate into the galactic community and shared their own discoveries. The mission led to new alliances and advancements in technology, showcasing the importance of exploration and cooperation in the galaxy.

# The Interstellar Symphony
# In a future where music was a universal language, an interstellar orchestra traveled the galaxy, sharing their melodies with diverse civilizations. The orchestra, known as the Harmonic Voyager, performed concerts on various planets, using their music to foster peace and understanding. One day, they arrived at a planet where the inhabitants communicated solely through complex sound patterns. The orchestra faced the challenge of understanding and integrating with the planet's musical language. With the help of local musicians, they composed a symphony that resonated with the planet's unique frequencies. The performance was a profound experience, bridging the gap between cultures and creating a lasting bond between the orchestra and the planet's inhabitants. The Harmonic Voyager continued their journey, leaving behind a legacy of harmony and connection across the stars.

# The Starlight Colony
# On a distant planet known as Starlight, a group of settlers established a new colony in search of a fresh start. The planet's environment was harsh, with extreme temperatures and frequent storms. The settlers, led by Commander Ava Grant, faced numerous challenges as they built their new home. They relied on advanced technology to create a sustainable habitat and cultivate the land. As they adapted to their new environment, they discovered remnants of an ancient alien civilization that had once inhabited the planet. The discovery revealed advanced technology and knowledge that helped the settlers improve their living conditions. However, they also uncovered warnings about a looming threat that had driven the aliens away. Commander Grant and her team had to prepare for the impending danger while ensuring the colony's survival. Their efforts led to the successful establishment of the Starlight Colony, a beacon of resilience and ingenuity in the face of adversity.

# The Solar Wanderer
# In the year 3100, humanity had developed the technology to travel between star systems using solar sails. The *Solar Wanderer*, a sleek vessel equipped with cutting-edge sails, embarked on a mission to explore uncharted regions of the galaxy. The crew, led by Captain Elias Drake, encountered a series of mysterious phenomena, including strange energy readings and uncharted celestial objects. Their journey took them to a distant star system where they discovered an ancient alien outpost. The outpost contained remnants of a civilization that had once explored the galaxy but had vanished. The crew deciphered the alien technology and learned about the outpost's purpose: to monitor and protect a crucial cosmic gateway. The gateway, if activated, could reshape the galaxy's gravitational fields. The crew had to ensure that the gateway remained secure while unraveling the secrets of the alien civilization. Their mission highlighted the importance of exploration and the preservation of cosmic knowledge.

# The Sentient Planet
# On a remote world known as Gaia, scientists discovered that the planet itself possessed a form of sentience. The planet communicated through subtle shifts in its environment and energy fields. Dr. Nina Patel, a leading researcher, studied the planet's behavior and interactions with its inhabitants. The sentience of Gaia was revealed to be a result of an ancient technology that had evolved over millennia. The planet's consciousness had developed a deep connection with its ecosystems, guiding and nurturing life. When a mining corporation threatened to exploit Gaia's resources, Dr. Patel and her team had to protect the planet and its unique sentience. They worked to raise awareness and implement sustainable practices to preserve Gaia's balance. The efforts succeeded in protecting the planet, and Gaia became a symbol of the harmony between technology and nature. The research also led to new insights into the potential for sentient worlds in the universe.

# The Paradox Machine
# In a futuristic society where time travel was a reality, Dr. Ethan Hayes, a renowned physicist, invented a device capable of creating time loops. The Paradox Machine allowed users to revisit and alter specific moments in history. However, its use led to unforeseen consequences, including disruptions in the space-time continuum. Dr. Hayes and his team discovered that their alterations were creating dangerous paradoxes that threatened the stability of reality. They faced the challenge of correcting the anomalies while preventing further damage. Their efforts involved intricate calculations and precise adjustments to restore the timeline. As they resolved the paradoxes, they realized the importance of preserving the integrity of time and the delicate balance required to maintain it. The Paradox Machine was decommissioned, and its technology was safeguarded to prevent future misuse. The experience underscored the potential risks and responsibilities associated with time manipulation.

# The Cosmic Architects
# A team of cosmic architects was tasked with designing a new galaxy to accommodate expanding civilizations. Using advanced technology, they created stars, planets, and ecosystems, carefully balancing the elements to ensure stability. As they worked on their grand project, they encountered unexpected challenges, including cosmic anomalies and competing interests from various civilizations. The architects had to collaborate and innovate to address these issues while maintaining the integrity of their designs. Their efforts culminated in the creation of a thriving galaxy, filled with diverse worlds and harmonious ecosystems. The project became a testament to the creativity and ingenuity of the cosmic architects, inspiring future generations to explore and appreciate the wonders of the universe.

# ###